In [2]:
import os 
os.chdir("../")
os.getcwd()

'/home/googlyji/ml_projects/dl_projects/brain_tumor/brain-tumor-classification'

In [3]:
from src.brain_tumor_classification.logger import logging 
from src.brain_tumor_classification.exception import Custom_Exception

In [8]:
import tensorflow as tf 
from tensorflow import keras 
from dataclasses import dataclass
from pathlib import Path 

In [16]:
import sys

In [11]:
from src.brain_tumor_classification.constants import * 
from src.brain_tumor_classification.config import * 
from src.brain_tumor_classification.utils.common import *

In [19]:
#entity 
@dataclass
class evaluation:
    training_data: Path
    model_path: Path
    image_size: list


In [29]:

class Configuration_manager:

    def __init__(self,config_path=CONFIG_PATH,param_path=PARAMS_PATH):
        config_path = Path(config_path)
        param_path = Path(param_path)
        self.config=read_yaml(config_path)
        self.params=read_yaml(param_path)

        create_directories([self.config.artifacts_root])


    def evluation_config(self)->evaluation:
        config=self.config.training

        evaluation_stats=evaluation(
            training_data=Path("artifacts/data_ingestion/files/bain_tumor/Testing"),
            model_path=config.trained_model_path,
            image_size=self.params.Image_size
        )

        return evaluation_stats


In [35]:
class Evaulation_start:
    try:
        def __init__(self,config:evaluation):
            self.config=config


        def train_images(self):
            self.train_ds=tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255.,
                    validation_split=0.2,
                    horizontal_flip=True,rotation_range=0.2,zoom_range=0.2)
        
            self.val_ds=tf.keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255.,
                validation_split=0.2
            )
        
            self.train_img=self.train_ds.flow_from_directory(self.config.training_data,target_size=(224,224),
                                                        class_mode="categorical",batch_size=12,seed=1337,
                                                        subset="training")
        
            self.valid_img=self.val_ds.flow_from_directory(self.config.training_data,target_size=(224,224),
                                                    class_mode="categorical",batch_size=12,seed=1337,
                                                    subset="validation")
        
    
        def load_model(self):
            return tf.keras.models.load_model(self.config.model_path)
    


        def evaluation(self):
            self.model=self.load_model()
        
            self.score=self.model.evaluate(self.valid_img)

    
        def evaluate_score(self):
            return f"loss is {self.score[0]} and accuracy is {self.score[1]}"
        

    except Exception as e:
        raise(Custom_Exception(e,sys))
    

        

In [36]:
a=Configuration_manager()
eval_=a.evluation_config()
z=Evaulation_start(eval_)
z.train_images()
z.evaluation()
z.evaluate_score()


Found 1049 images belonging to 4 classes.


Found 262 images belonging to 4 classes.
22/22 [==============================] - 23s 1s/step - loss: 0.9195 - accuracy: 0.6489


AttributeError: 'Evaulation_start' object has no attribute 'save_score'